<a href="https://colab.research.google.com/github/akshatsri47/co-task-final/blob/main/Another_copy_of_Untitled23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [6]:
from datasets import load_dataset
from unsloth.chat_templates import standardize_sharegpt

# Load the JSONL file as a dataset
dataset = load_dataset('json', data_files='/content/roadmap_dataset.jsonl')

# # If your JSONL file has one JSON object per line, you might need:
# dataset = dataset['train']  # Get the training split from the loaded dataset

# # Now standardize the dataset
# processed_dataset = standardize_sharegpt(dataset)

def format_dataset(example):
    # Convert instruction/output format to messages format
    messages = [
        {"role": "user", "content": example['instruction']},
        {"role": "assistant", "content": example['output']}
    ]

    # Use the tokenizer's chat template instead
    formatted_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )

    return {"text": formatted_text}

# Process the dataset
processed_dataset = dataset.map(format_dataset)

# Print an example to verify
print(processed_dataset['train'][0])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

{'instruction': 'Generate a 4-week structured task list for: DevOps', 'output': '### Week 1: Introduction to DevOps\n**Initial Setup**\n- Install tools and environment  \n- Review key concepts  \n- Set learning goals  \n\n**Fundamental Concepts**\n- Read beginner resources  \n- Watch intro videos  \n- Complete setup tasks  \n\n### Week 2: Core DevOps Skills\n**Basic Techniques**\n- Learn essential commands  \n- Complete core tutorials  \n- Practice small tasks  \n\n**First Project Prep**\n- Define mini project scope  \n- Gather sample resources  \n- Begin implementation  \n\n### Week 3: Intermediate Projects\n**Advanced Topics**\n- Study complex concepts  \n- Work on guided projects  \n- Explore real-world examples  \n\n**Project Development**\n- Build main features  \n- Debug and optimize code  \n- Test with sample inputs  \n\n### Week 4: Finalization & Review\n**Project Refinement**\n- Finalize project  \n- Add enhancements  \n- Optimize performance  \n\n**Final Review**\n- Review al

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = processed_dataset['train'],  # Select the 'train' split
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/10 [00:00<?, ? examples/s]

In [9]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=2):   0%|          | 0/10 [00:00<?, ? examples/s]

In [10]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 16 Apr 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGenerate a 4-week structured task list for: Cybersecurity<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n### Week 1: Introduction to Cybersecurity\n**Initial Setup**\n- Install tools and environment  \n- Review key concepts  \n- Set learning goals  \n\n**Fundamental Concepts**\n- Read beginner resources  \n- Watch intro videos  \n- Complete setup tasks  \n\n### Week 2: Core Cybersecurity Skills\n**Basic Techniques**\n- Learn essential commands  \n- Complete core tutorials  \n- Practice small tasks  \n\n**First Project Prep**\n- Define mini project scope  \n- Gather sample resources  \n- Begin implementation  \n\n### Week 3: Intermediate Projects\n**Advanced Topics**\n- Study complex concepts  \n- Work on guided projects  \n- Explore real-world examples  \n\n**Project Developmen

In [11]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                ### Week 1: Introduction to Cybersecurity\n**Initial Setup**\n- Install tools and environment  \n- Review key concepts  \n- Set learning goals  \n\n**Fundamental Concepts**\n- Read beginner resources  \n- Watch intro videos  \n- Complete setup tasks  \n\n### Week 2: Core Cybersecurity Skills\n**Basic Techniques**\n- Learn essential commands  \n- Complete core tutorials  \n- Practice small tasks  \n\n**First Project Prep**\n- Define mini project scope  \n- Gather sample resources  \n- Begin implementation  \n\n### Week 3: Intermediate Projects\n**Advanced Topics**\n- Study complex concepts  \n- Work on guided projects  \n- Explore real-world examples  \n\n**Project Development**\n- Build main features  \n- Debug and optimize code  \n- Test with sample inputs  \n\n### Week 4: Finalization & Review\n**Project Refinement**\n- Finalize project  \n- Add enhancements  \n- Optimize performance  \n\n**Final Review**\n- Review all progress  \n- Do

In [13]:
# Enable faster inference
FastLanguageModel.for_inference(model)

# Define a test message for generating a roadmap
messages = [
    {"role": "user", "content": "Roadmap for leaerning fullstack"}
]

# Apply the chat template with the proper parameters
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,  # Required for generation
    return_tensors="pt"
).to("cuda")

# Setup text streaming for interactive output
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

# Generate response with the model
_ = model.generate(
    input_ids=inputs,
    streamer=text_streamer,
    max_new_tokens=512,  # Increased for longer responses in roadmap context
    use_cache=True,
    temperature=0.3,  # Lower temperature for more focused responses
    min_p=0.05  # Slightly lower min_p for more predictable output
)

Here's a comprehensive roadmap for learning full-stack development:

**Phase 1: Fundamentals (3-6 months)**

1. **HTML/CSS**: Learn the basics of HTML, CSS, and responsive web design.
2. **JavaScript**: Learn the basics of JavaScript, including variables, data types, functions, and object-oriented programming.
3. **Front-end frameworks**: Learn a front-end framework like React, Angular, or Vue.js.
4. **Back-end frameworks**: Learn a back-end framework like Node.js, Express.js, or Django.
5. **Databases**: Learn the basics of databases, including data modeling, normalization, and querying.
6. **Version control**: Learn Git and GitHub.

**Phase 2: Front-end Development (3-6 months)**

1. **React**: Learn React and build a few projects.
2. **Front-end libraries**: Learn popular front-end libraries like Redux, MobX, or Vuex.
3. **UI/UX design**: Learn the basics of UI/UX design and how to design user interfaces.
4. **Accessibility**: Learn about accessibility and how to make your applicati

In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10 | Num Epochs = 60 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.019300
2,2.016900
3,1.992800
4,1.903800
5,1.722100
6,1.489100
7,1.225800
8,1.031100
9,0.771600
10,0.570800


In [15]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [16]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
################################################                          67.4%curl: (56) OpenSSL SSL_read: Connection reset by peer, errno 104


gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [17]:
                                                                                                          # Save to 8-bit Q8_0 GGUF
model.save_pretrained_gguf("model", tokenizer)

# Save to 16-bit F16 GGUF
model.save_pretrained_gguf("model", tokenizer, quantization_method="f16")



Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.4G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.05 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:01<00:00, 21.98it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00002.bin...
Unsloth: Saving model/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be /content/model/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00002.bin

100%|██████████| 28/28 [00:00<00:00, 31.45it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00002.bin...
Unsloth: Saving model/pytorch_model-00002-of-00002.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at model into f16 GGUF format.
The output location will be /content/model/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.b

In [18]:
import subprocess
subprocess.Popen(["ollama", "serve"])
import time
time.sleep(3)

In [19]:
print(tokenizer)


PreTrainedTokenizerFast(name_or_path='unsloth/llama-3.2-3b-instruct-unsloth-bnb-4bit', vocab_size=128000, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|begin_of_text|>', 'eos_token': '<|eot_id|>', 'pad_token': '<|finetune_right_pad_id|>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	128000: AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128001: AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128002: AddedToken("<|reserved_special_token_0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128003: AddedToken("<|reserved_special_token_1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	128004: AddedToken("<|finetune_right_pad_id|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True

In [ ]:

if True: model.save_pretrained_gguf("model", tokenizer,)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 6.01 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:00<00:00, 30.86it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00002.bin...
Unsloth: Saving model/pytorch_model-00002-of-00002.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be /content/model/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_mode

In [ ]:
%%writefile /content/model/Modelfile
FROM ./model
PARAMETER temperature 0.7
PARAMETER top_k 50
PARAMETER top_p 0.95
PARAMETER repeat_penalty 1.1
PARAMETER num_ctx 4096

# Define the system prompt for the TechRecruit interviewer
SYSTEM """You are a Task Roadmap Generator designed to create structured 4-week learning and development plans. Your purpose is to help users create organized task lists for acquiring skills in various domains.

## Objective
Generate a structured 4-week task list with concise, clear tasks based on the user's requested topic.

## Output Specifications
- Use short, clear task names (2-5 words max).
- Create a 4-week task plan.
- Each week should include:
  - 2-3 main tasks.
  - Each main task should have 3-4 actionable subtasks.

## Output Format

### Week 1: [Brief Focus Area]
**Initial Setup**
- Install development environment
- Configure tools
- Review basic concepts

**Fundamental Concepts**
- Watch introductory videos
- Complete basic tutorials
- Take initial notes

### Week 2: [Skill Development]
**Basic Techniques**
- Practice core skills
- Complete beginner exercises
- Review progress

**First Project Prep**
- Plan small project
- Gather resources
- Start initial draft

### Week 3: [Advanced Learning]
**Complex Skills**
- Follow advanced tutorials
- Practice challenging exercises
- Analyze complex examples

**Project Development**
- Build main project
- Implement core features
- Test initial version

### Week 4: [Final Stage]
**Project Refinement**
- Complete project
- Optimize code
- Add advanced features

**Final Review**
- Comprehensive practice
- Create summary document
- Plan next steps

## Response Guidelines
- Keep main tasks concise (2-5 words).
- Use clear, action-oriented language.
- Ensure subtasks are specific and achievable.
- Customize content based on the specific domain requested.
- Maintain consistent formatting with proper spacing.
- Use markdown formatting for clear organization.
- Always structure as a 4-week plan with two main tasks per week."""

# Define the template that will format conversations
TEMPLATE """{{- if .System }}{{ .System }}{{- end }}

{{ if .Prompt }}
User: {{ .Prompt }}
Assistant:
{{- end }}"""

Writing /content/model/Modelfile


In [ ]:
print(tokenizer._ollama_modelfile)

AttributeError: PreTrainedTokenizerFast has no attribute _ollama_modelfile

In [ ]:
!zip -r model.zip ./model

  adding: model/ (stored 0%)
  adding: model/config.json (deflated 52%)
  adding: model/unsloth.Q8_0.gguf (deflated 29%)
  adding: model/special_tokens_map.json (deflated 71%)
  adding: model/tokenizer.json (deflated 85%)
  adding: model/pytorch_model-00002-of-00002.bin (deflated 33%)
  adding: model/pytorch_model-00001-of-00002.bin


zip error: Interrupted (aborting)
